# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

## Задание. 

1. hit rate at k
2. money_precision_at_k
3. recall_at_k
4. money_recall_at_k
5. *average_precision для ранжирования по примеру с картинки лекции
6. map@k
7. *NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции
8. Mean Reciprocal Rank

In [48]:
import numpy as np
import pandas as pd

recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]

prices_recommended = [100, 90, 10, 450, 50, 37, 99, 120, 34, 100]
prices_bought = [110, 190, 100, 450]

### 1. hit_rate@K

In [49]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
   
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [50]:
hit_rate_at_k(recommended_list, bought_list)

1

### 2. money_precision@k

In [51]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991] 
            #  [0, 0, 100, 450] 

prices_recommended = [100, 90, 10, 450, 50, 37, 99, 120, 34, 100]
prices_bought = [110, 190, 100, 450]

In [52]:
money_precision = (1*100 + 0*90 + 0*10 + 1*450+ 0*50 + 0*37 + 0*99 + 0*120 + 0*34 + 0 * 100)/(100+ 90+ 10+ 450+ 50+ 37+ 99+ 120+ 34+ 100)
money_precision

0.5045871559633027

In [53]:
money_precision_at_5 = (1*100 + 0*90 + 0*10 + 1*450+ 0*50)/(100+ 90+ 10+ 450+ 50)
money_precision_at_5

0.7857142857142857

In [54]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    flags = np.array(np.isin(recommended_list, bought_list), dtype=np.int32)
    
    result = flags * prices_recommended
    m_precision = result.sum() / prices_recommended.sum()
    
    return m_precision

In [55]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.7857142857142857

### 3. recall@k

In [56]:
recall_at_5 = (1 + 0 + 0 + 1 + 0)/(4) 
recall_at_5

0.5

In [57]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list) 
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [58]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

### 4. money_recall@k

In [59]:
money_recall_at_k = (1*100 + 0*90 + 0*10 + 1*450+ 0*50)/(110 + 190 + 100 + 450)
money_recall_at_k

0.6470588235294118

In [60]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.array(np.isin(recommended_list, bought_list), dtype=np.int32)
    result = flags * prices_recommended
    recall = result.sum() / prices_bought.sum()
    
    return recall

In [61]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=4)

0.6470588235294118

### 5. *average_precision (для ранжирования по примеру с картинки лекции)

In [62]:
def precision_at_k(recommended_list, bought_list, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.array(np.isin(recommended_list, bought_list), dtype=np.int32)

    m_precision = flags.sum() / len(recommended_list)
    
    return m_precision

In [63]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0

    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [64]:
ap_k(recommended_list, bought_list, k=5)

0.75

### Для примера из урока

<img src="ap_k.png">

In [65]:
# Посчитаем сначала вручную.
ap_4 = 1/4*(1/1+2/3+3/4+4/6)
ap_4

0.7708333333333333

In [66]:
ap_recommended_list = [143, 156, 521, 32, 27, 991, 3345, 533, 11, 43]
ap_bought_list = [521, 32, 143, 991]

ap_k(ap_recommended_list, ap_bought_list, k=8)

0.7708333333333333

### 6. MAP@k (Mean Average Precision@k)  

$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$
  
|U| - кол-во юзеров

In [67]:
def map_k(recommendeds, boughts, k=5):
    items_count = len(recommendeds)
    ap_k_list = []
    for i in range(items_count):
        ap_k_list.append(ap_k(recommendeds[i], boughts[i], k))
    
    ap_k_list = np.array(ap_k_list)
    result = ap_k_list.sum()/items_count
    return result

Чтобы не усложнять задачу. Просто возьмем список с рекомендациями (recommended_list и bought_list) и, продублирую несколько раз.  

In [68]:
recommendeds = [recommended_list,recommended_list,recommended_list] 
boughts = [bought_list,bought_list,bought_list]

map_k(recommendeds, boughts, k=5)

0.75

### 7. *NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции

In [69]:
DCG_5 =1/5*(1+ 0/np.log(2)+ 0/np.log(3) + 1/np.log(4)+ 0/np.log(5))  

ideal_DCG_5 =1/5*(1 + 1/np.log(2) + 1/np.log(3) + 1/np.log(4)+ 1/np.log(5))

print(f'DCG_5: {DCG_5}')
print(f'ideal_DCG_5: {ideal_DCG_5}')
print('-'*30)
print(f'NDCG_k: {DCG_5/ideal_DCG_5}')

DCG_5: 0.34426950408889634
ideal_DCG_5: 0.939123344503979
------------------------------
NDCG_k: 0.36658603590642364


In [70]:
def NDCG_k(recommended_list, bought_list, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list) 
    
    if sum(flags) == 0:
        return 0

    dcg = 1 if flags[0] == True else 0 
    idcg = 1

    for i in range(1, k):
        if flags[i] == True:
            dcg += 1/np.log(i+1)
        idcg += 1/np.log(i+1)

    result = (dcg / idcg)
    
    return result    

In [71]:
NDCG_k(recommended_list, bought_list, k=5)

0.3665860359064237

### Для примера из урока.

В примере использованы 3-и уровня релевантности. 

1. Не релевантно,
2. В целом релевантно,
3. Полное совпадение.

<img src='comulative_gain.png'>

<img src='Discounted_comulative_gain.png'>

Здесь не вполне понятно как оценивать все 3-и уровня релевантности.  Необходим какой-то более сложный пример чем  просто номера товаров.

1. Вообще не угадал
2. Например угадал категорию товара но, не угадал марку.
3. Все совпало.

In [73]:
DCG_3 =1/3*(3+ 0/np.log(2)+ 2/np.log(3) + 1/np.log(4))  

ideal_DCG_3 =1/3*(3 + 3/np.log(2) + 3/np.log(3) + 3/np.log(4))

print(f'DCG_3: {DCG_3}')
print(f'ideal_DCG_3: {ideal_DCG_3}')
print('-'*30)
print(f'NDCG_3: {DCG_3/ideal_DCG_3}')

DCG_3: 1.847275324566052
ideal_DCG_3: 4.074281787960283
------------------------------
NDCG_3: 0.45339900888172435


### 8. Mean Reciprocal Rank

$$MRR= \frac{1}{5}\cdot\big(\frac{1}{3}+\frac{1}{4}\big) = 0.1166$$

In [74]:
Mrr= 1/5*(1/3+1/4)
Mrr

0.11666666666666665

In [75]:
def reciprocal_rank(recommended_list, bought_list, k=5): # здесь еще к-во добавить нужно
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    ranks=0.
    
    for item_rec in recommended_list:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
                # print(i+1)

    return ranks / len(recommended_list)

In [76]:
reciprocal_rank(recommended_list, bought_list, k=5)

0.11666666666666665